In [ ]:
# Getting started

In [1]:
push!(LOAD_PATH, "../../../ExoJulia/")

3-element Array{ByteString,1}:
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/local/share/julia/site/v0.4"
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/share/julia/site/v0.4"      
 "../../../ExoJulia/"                                                                

In [ ]:
Pkg.add("LsqFit")

In [2]:
using ExoJulia
using PyPlot
using LsqFit

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
include("utils.jl")
include("orbital_utils.jl")
include("rv.jl")

agol_periodogram (generic function with 1 method)

### Read-in Mystery RV data

In [ ]:
numbers = readdlm("mystery_planet.txt");

In [ ]:
time = numbers[:,1];
rv = numbers[:,2];
err = numbers[:,end];

In [ ]:
plot(time, rv, "o", ms=2.0)
xlabel("Time")
ylabel("RV")

## Period Fitting

### Agol Method

In [ ]:
periods = linspace(1.0, 365.0, 1000)

In [ ]:
p = collect(periods);

In [ ]:
@time best_period = agol_periodogram(numbers, p)
# [time rv err]

## RV Fitting

### Matrix Approach

In [ ]:
using Optim

In [4]:
function WH_W(err::Array{Float64,1})
    # Compose Wright & Howard  matrix
    W = zeros(Float64,length(err), length(err))
    for i=1:length(err)
        W[i,i] = 1./err[i]^2
    end 
    return W
end 

WH_W (generic function with 1 method)

In [ ]:
@time W = WH_W(err);

In [5]:
function f_from_t(P::Float64, ecc::Float64, t::Float64, tp::Float64)
    f_from_M(ecc, mean_anomaly(P, t, tp))
end 

f_from_t (generic function with 1 method)

In [6]:
function WH_F(P::Float64, ecc::Float64, t::Array{Float64, 1}, tp::Float64; Nplanets::Int=1)
    # Compute Wright & Howard F matrix
    
    # Allocate matrix
    F = zeros(Float64, (2*Nplanets+2), length(t))
    
    # Fill matrix
    for i=1:Nplanets
        for j=1:length(t)
            f = f_from_t(P, ecc, t[j], tp)
            F[2*i - 1,j] = cos(f)
            F[2*i, j] = sin(f)
        end 
    end 
    
    # 
    for j=1:length(t)
        F[end-1, j] = 1.0
        F[end, j] = t[j] - t[1]
    end 
    return F
end

WH_F (generic function with 1 method)

In [ ]:
F = WH_F(best_period, 0.1, time, 1.0)

In [7]:
function WH_eps(F::Array{Float64, 2}, W::Array{Float64, 2})
    # Compute Wright & Howard epsilon matrix
    return inv(F * W * (F'))
end 

WH_eps (generic function with 1 method)

In [ ]:
WH_eps(F, W)

In [8]:
function WH_Beta(RV::Array{Float64, 2}, F::Array{Float64, 2}, W::Array{Float64, 2})
    # Compute Wright & Howard Beta vector
    return RV * W * (F') * WH_eps(F, W)
end 

WH_Beta (generic function with 1 method)

In [ ]:
B = WH_Beta(rv', F, W)

In [9]:
function rv_forward(P::Float64, ecc::Float64, tp::Float64, t::Array{Float64,1}, rv::Array{Float64,1}, err::Array{Float64,1}; Nplanets::Int=1)
    # Calculates model RV given linear params: [h, c, v0, d, tp]
    
    # Allocate 
    rv_mod = zeros(Float64, Nplanets, length(t))
    
    # Calculate Wright & Howard Beta Vector, B = [h_i, c_i, ..., h_n, c_n, v0, d] for n planets
    B = WH_Beta(rv', WH_F(P, ecc, t, tp), WH_W(err))
    
    # Loop over planets and observations, calculating model rv points
    for i=1:Nplanets
        for j=1:length(t)
            f = f_from_t(P, ecc, t[j], tp)
            rv_mod[i,j] = v_rad_lin(B[i,1], f, B[i,2], B[i,end-1], t[j], t[1], B[i,end])
        end 
    end 
    
    return rv_mod, B
end 

rv_forward (generic function with 1 method)

In [10]:
function loglike(data, model, err)
    # Chi^2
    ll = 0.0;
    for i=1:length(data)
        ll -= 0.5 * (data[i] - model[i])^2 / (err[i])^2;
    end
    return ll
end 

loglike (generic function with 1 method)

### Grid-search

In [ ]:
ecc = linspace(0.0, 0.99, 1000);
tp = linspace(time[1], time[end], 1000);
period = best_period;

ll_min = -1.0e20;
B_best = nothing;
ecc_best = nothing;
tp_best = nothing;
rv_best = nothing;
for i=1:length(ecc)
    for j=1:length(tp)
        rv_mod, B = rv_forward(period, ecc[i], tp[j], time, rv, err);
        ll = loglike(rv, rv_mod, err);
        if ll > ll_min
            B_best = B;
            ll_min = ll;
            ecc_best = ecc[i];
            tp_best = tp[j];
            rv_best = rv_mod;
        end 
    end 
end         

In [ ]:
ecc_best

In [ ]:
function rv_grid(rv_data, time, err; N::Int=1000)    
    ecc = linspace(0.0, 0.99, N);
    tp = linspace(time[1], time[end], N);
    period = best_period;

    ll_min = -1.0e20;
    B_best = nothing;
    ecc_best = nothing;
    tp_best = nothing;
    rv_best = nothing;
    for i=1:length(eccentricity)
        for j=1:length(tp)
            rv_mod, B = rv_forward(period, ecc[i], tp[j], time, rv, err);
            ll = loglike(rv, rv_mod, err);
            if ll > ll_min
                B_best = B;
                ll_min = ll;
                ecc_best = ecc[i];
                tp_best = tp[j];
                rv_best = rv_mod;
            end 
        end 
    end 
    
end 

In [ ]:
plot(time_fold, rv_best', "o", ms=5)
plot(time_fold, rv, "o")
xlabel("Days")
ylabel("RV")

In [11]:
function rv_loglike(rho)
    #rho = [period, ecc, tp]
    
    # hard bounds
    if rho[1] < 0.0
        return Inf
    end 
    if rho[2] >= 1.0
        return Inf
    end 
    if rho[2] < 0.0 
        return Inf
    end
    
    # call forward model 
    model, B = rv_forward(rho[1], rho[2], rho[3], time, rv, err);
    
    # Chi^2
    return -loglike(rv, model, err);
end 

rv_loglike (generic function with 1 method)

### Optim `optimize`

In [12]:
using Optim

In [ ]:
p0 = [best_period, 0.0, mean(time)];
@time optimum = optimize(rv_loglike, p0, autodiff=true)

In [ ]:
MLE = optimum.minimum

### `curve_fit()`

In [ ]:
numbers = readdlm("mystery_planet.txt");

In [13]:
function rv_curve_forward(t, p::Vector)
        
    # call forward model 
    model, B = rv_forward(p[1], p[2], p[3], t, rv, err);
        
    return reshape(model, length(model))
    
end

rv_curve_forward (generic function with 1 method)

In [ ]:
p0 = [best_period, 0.0, mean(time)];
@time fit3 = curve_fit(rv_curve_forward, time, rv, 1.0./err.^2, [best_period, 0.0, mean(time)]);

In [ ]:
fit3.param

In [14]:
function solve_rv(data::Array{Float64, 2}; p0=[nothing, nothing, nothing], alg::AbstractString="cf")
    """ This function takes a 2D (Nx3) array of RV data, where N is the number of data points with columns
    of time, RV, and error, and returns the best fitting period, eccentricity, and time of periastron.
    
    p0 = [period, ecc, tp]
    """
    
    # Unpack RV data (make global?)
    time = data[:,1];
    rv = data[:,2];
    err = data[:,end];
    
    # Set initial parameters if not specified
    p = [0.0, 0.0, 0.0]
    if p0[1] == nothing
        # Use Agol Periodogram for initial period guess
        periods = collect(linspace(minimum(time[2:end] - time[1:end-1]), time[end]-time[1], 10000))
        p[1] = agol_periodogram(numbers, periods)
    else
        p[1] = p0[1]
    end 
    if p0[2] == nothing
        # Use random ecc [0,1) for initial guess
        p[2] = rand()
    else
        p[2] = p0[2]
    end 
    if p0[3] == nothing
        # Use random time drawn from observed grid
        p[3] = rand(time)
    else
        p[3] = p0[3]
    end
    
    # Run solver using either curve_fit() or optimize()
    if alg == "cf"
        # Use curve_fit() to fit 
        fit = curve_fit(rv_curve_forward, time, rv, 1.0./err.^2, p);
        pbest = fit.param
    elseif alg == "opt"
        # Use optimize to fit
        optimum = optimize(rv_loglike, p0, autodiff=true)
        pbest = optimum.minimum
    else
        print("Choose alg = 'cf' or 'opt'")
        return 
    end 
    
    return pbest   
end 

solve_rv (generic function with 1 method)

In [21]:
numbers = readdlm("mystery_planet.txt");
time = numbers[:,1];
rv = numbers[:,2];
err = numbers[:,end];

In [24]:
# testing curve fit
@time pbest = solve_rv(numbers, p0 = [116.0, 0.0, mean(time)], alg="cf")

3-element Array{Float64,1}:
   116.682   
     0.399921
 15058.2     

0.007656 seconds (4.30 k allocations: 3.171 MB)


In [25]:
# testing optimize
@time pbest = solve_rv(numbers, p0 = [116.0, 0.0, mean(time)], alg="opt")

3-element Array{Float64,1}:
   116.682  
     0.40072
 15058.2    

0.022698 seconds (10.15 k allocations: 7.972 MB)
